In [1]:
import csv
import numpy as np
import pandas as pd

import data
# Word2vec
from gensim.models import Word2Vec
from gensim.utils import tokenize
from unidecode import unidecode

2022-12-04 23:56:01.806389: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 23:56:01.910430: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 23:56:01.911179: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
train_data = pd.read_csv("data/train.csv")

In [3]:
# embedding dimension
n_dim = 32

In [4]:
sentences = [list(tokenize(s, deacc=True, lower=True)) for s in train_data['text']]
print(sentences[:5])

[['rt', 'refarcir', 'macron', 'ans', 'nom', 'prepare'], ['populaire'], ['faut', 'degager', 'cingle'], ['enseignants', 'mettre', 'prescriptions', 'president', 'republique', 'faut', 'croire', 'president', 'donner', 'periode', 'compte', 'mesures', 'appliquees', 'grever'], ['mafieuse', 'oppressive', 'macron']]


In [5]:
model = Word2Vec(vector_size=n_dim, window=20, min_count=1, workers=7)
model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=5)

(15075704, 16375635)

In [6]:
embeddings = np.zeros((train_data.shape[0], n_dim))

In [7]:
for idx in range(train_data.shape[0]):
    for s in train_data.loc[idx, "text"].split():
        try:
            embeddings[idx] += model.wv[unidecode(s).lower()]
        except:
            pass
            # print(idx, unidecode(s).lower())

In [8]:
df_new = pd.concat([train_data, pd.DataFrame(embeddings)], axis=1)

In [9]:
display(df_new)

,text,retweets_count,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,...,22,23,24,25,26,27,28,29,30,31
0,rt refarcir macron ans nom prépare,3,0,3682,453535,3628,[],[],0,[],...,10.507610,-5.182530,5.039705,0.243909,-0.722506,-3.429304,-2.017067,-7.079484,-3.715013,-3.219591
1,populaire,0,0,86,1016,284,[],[],0,[],...,1.871287,-2.451910,-0.967159,1.582584,-4.310072,1.636796,-0.166424,-1.586774,1.489932,5.812188
2,faut dégager cinglé,3,1,1944,28234,1995,[],[],0,[],...,2.911065,0.198195,-0.171330,6.812440,-8.514426,2.015868,-0.907253,-5.232990,-4.939513,-4.880474
3,enseignants mettre prescriptions président rép...,0,0,1,1072,0,[],['https://t.co/rytlted08g'],0,[],...,25.838896,-11.164051,-9.657174,12.524952,-9.429497,0.700722,-14.425915,-15.583109,-16.634422,-15.014481
4,mafieuse oppressive macron,0,0,13957,25311,10841,[],[],0,[],...,1.914337,-0.734419,2.467783,-1.053201,-0.705008,1.463999,0.017384,-3.423769,-1.934715,-3.634649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353964,gonflette tour raciste frustré,0,0,34,1509,55,[],['https://t.co/pma33zhslx'],0,[],...,3.624929,-0.057951,1.339173,0.947747,-7.052218,7.172041,1.601568,-5.575241,0.567228,0.198913
353965,france caste crapuleuse encadrée gangsters irr...,0,0,89,11166,127,[],[],0,[],...,5.892468,-5.939347,4.697102,6.414588,-2.712446,-3.570886,2.080191,-8.007005,-2.350684,-0.636648
353966,eric zemmour français berbère,3,0,1888,712,3086,[],[],0,[],...,9.015864,5.443816,14.853198,-4.084426,-5.767371,-1.956669,4.849006,-11.822286,3.734580,1.842272
353967,gauchistes dépression pq,0,0,139,486,320,[],[],0,[],...,0.600480,-4.860621,0.271188,3.219448,-3.280331,5.276660,4.050145,0.108980,-2.307451,-0.983604


In [10]:
file_path = f"data/embeddings/train_dim_{n_dim}.csv"

In [11]:
df_new.to_csv(file_path)

In [12]:
eval_data = pd.read_csv("data/evaluation.csv")

In [13]:
embeddings = np.zeros((eval_data.shape[0], n_dim))

In [14]:
for idx in range(eval_data.shape[0]):
    for s in eval_data.loc[idx, "text"].split():
        try:
            embeddings[idx] += model.wv[unidecode(s).lower()]
        except:
            pass
            # print(idx, unidecode(s).lower())

In [15]:
df_new = pd.concat([eval_data, pd.DataFrame(embeddings)], axis=1)

In [16]:
display(df_new)

,text,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,timestamp,...,22,23,24,25,26,27,28,29,30,31
0,descendue pourcentage vote,0,85,4442,327,[],[],0,"['presidentielle2022', 'hidalgo']",1647607994000,...,2.085435,-0.172552,-0.816812,-1.650321,-5.758308,0.735596,-1.244580,-1.031486,-2.125419,-0.858241
1,photo demande arbre généalogique,0,427,33282,443,[],[],0,[],1647601275000,...,3.836601,-6.041139,0.187394,-3.979446,-3.067158,0.456576,-1.206806,1.008825,0.886927,-2.082169
2,dossiers promis poste gars vraiment fiable,6,1127,13111,1596,[],[],0,['macron'],1646900684000,...,5.641206,-1.900767,-2.439815,1.579042,-2.661772,4.519307,-2.097608,-5.136146,-7.989482,-4.772130
3,lol cas faut second tour macron sortir définit...,2,1699,25760,2036,[],['https://t.co/7ox54d8dlm'],0,[],1647161294000,...,31.396628,6.713350,9.027090,10.195317,-43.448731,10.855514,-14.312541,-19.284498,-7.243942,-12.737749
4,zemmour putain discours propagandiste littéral...,0,249,20718,369,[],"['https://t.co/tsw9cg6v9j', 'https://t.co/t0j9...",0,[],1647724874000,...,15.976246,-10.148871,6.624080,15.974797,-24.987494,21.552225,1.351635,-18.951526,-5.387153,-11.251478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117985,balais inexcusable vote veut,0,4,194,64,[],[],0,[],1647103416000,...,3.387970,-2.574382,0.998942,1.068765,-6.973216,-0.311030,-1.958204,-7.230376,-1.591602,-2.556813
117986,balayant rap borloo mâles blcs commis faute fo...,0,744,39489,894,[],['https://t.co/im5cs5ur1a'],0,[],1647490899000,...,22.645443,-10.426708,-0.820521,9.071038,-4.720715,-2.863846,-5.158474,-14.672155,-11.093511,-5.195723
117987,triste réalité covid tue malades macron veille...,12,924,2270,159,[],[],0,[],1647522384000,...,15.034510,-9.570096,9.302993,10.834377,1.473109,1.424288,1.065702,-15.878493,-9.314908,-17.956211
117988,excellente idée pouvez réussir démarche crise ...,0,5,4,80,[],['https://t.co/xontgm06pb'],0,[],1647607262000,...,27.141048,-11.971247,7.714849,11.062534,-7.046428,-5.593230,-9.423055,-6.949477,-16.016979,-9.210331


In [17]:
file_path = f"data/embeddings/evaluation_dim_{n_dim}.csv"

In [18]:
df_new.to_csv(file_path)